# User Satisfaction Analysis

### Import Libraries and Modules

In [1]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
from dotenv import load_dotenv

# Define the path to the src directory
src_dir = os.path.abspath(os.path.join(os.getcwd(), '..', 'src'))
sys.path.insert(0, src_dir)

# Reload modules to ensure the latest versions are used
if 'data_loader' in sys.modules:
    del sys.modules['data_loader']
if 'user_engagement' in sys.modules:
    del sys.modules['user_engagement']
if 'user_experience' in sys.modules:
    del sys.modules['user_experience']
if 'user_satisfaction' in sys.modules:
    del sys.modules['user_satisfaction']

from data_loader import DataLoader
from user_engagement import UserEngagement
from user_experience import UserExperience
from user_satisfaction import UserSatisfaction

### Load Data from PostgreSQL and Initialize Models

In [2]:
# Load and clean data
loader = DataLoader()
df = loader.load_data("SELECT * FROM public.xdr_data")
cleaned_df = loader.clean_data(df)

# Initialize models
user_engagement = UserEngagement(cleaned_df)
user_experience = UserExperience(cleaned_df)
user_satisfaction = UserSatisfaction(cleaned_df, user_engagement, user_experience)


### Satisfaction scores

In [ ]:
# Calculate satisfaction scores
satisfaction_scores = user_satisfaction.calculate_satisfaction_scores()
print("Satisfaction scores calculated.")

# Get top satisfied customers
top_customers = user_satisfaction.get_top_satisfied_customers()
print("Top 10 satisfied customers:")
display(top_customers)

# Plot satisfaction scores
user_satisfaction.plot_satisfaction_scores()


### Regression Model

In [ ]:
# Build regression model
model, mse, r2 = user_satisfaction.build_regression_model()
print(f"Regression model built. MSE: {mse}, R-squared: {r2}")


### K-Means Clustering

In [ ]:
# Perform k-means clustering and plot results
user_satisfaction.plot_kmeans_results()

# Aggregate and plot scores by cluster
user_satisfaction.plot_aggregated_scores()

### Export table to database

In [ ]:
load_dotenv()
db_host = os.getenv('DB_HOST')
db_name = os.getenv('DB_NAME')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_port = os.getenv('DB_PORT')
# Export to PostgreSQL 
db_params = {
     'dbname': db_name,
     'user': db_user,
     'password': db_password,
     'host': db_host,
     'port': db_port
 }
engine = create_engine(f"postgresql://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['dbname']}")
select_query = user_satisfaction.export_to_postgres(engine)
select_query